**Function 4: Fast, but Inaccurate Modelling**

This example is for a particular business relying heavily on online sales. It can run very accurate calculations to figure out what is the optimal placement of their product across warehouses. Unfortunately, the calculations are extremely expensive (computationally) to run, so they can only do it once every two weeks. Instead, they propose using a machine learning model which approximates the solution quickly (in a few hours). The model has four hyper-parameters you need to tune, and the output corresponds to the difference between the expensive calculation, and the model. Since you are modelling a dynamical system, expect a lot of local optima!

In [3]:
import numpy as np

from get_init_data import get_inputs, get_outputs

f_num = 4

X_init = get_inputs(f_num)
y_init = get_outputs(f_num)
# print(X_init)
# print(y_init)

In [5]:
from get_query_data import get_inputs, get_outputs

X_q = get_inputs(f_num)
y_q = get_outputs(f_num)
# print(X_q)
# print(y_q)

[0.431257, 0.895431]
[0.821456, 0.390248]
[0.19432, 0.682014, 0.011249]
[0.129382, 0.503612, 0.777124, 0.041672]
[0.964301, 0.306598, 0.758164, 0.118309]
[0.338127, 0.072341, 0.600298, 0.901134, 0.244792]
[0.120936, 0.893025, 0.516204, 0.032114, 0.74219, 0.28831]
[0.024893, 0.310587, 0.498236, 0.865901, 0.702134, 0.139804, 0.603781, 0.003172]
[0.431257, 0.895431]
[0.821456, 0.390248]
[0.19432, 0.682014, 0.011249]
[0.129382, 0.503612, 0.777124, 0.041672]
[0.964301, 0.306598, 0.758164, 0.118309]
[0.338127, 0.072341, 0.600298, 0.901134, 0.244792]
[0.120936, 0.893025, 0.516204, 0.032114, 0.74219, 0.28831]
[0.024893, 0.310587, 0.498236, 0.865901, 0.702134, 0.139804, 0.603781, 0.003172]
[0.557814, 0.91103]
[0.423839, 0.177067]
[0.939684, 0.438755, 0.426575]
[0.439915, 0.427314, 0.95773, 0.471256]
[0.297493, 0.25993, 0.547059, 0.410727]
[0.848123, 0.282866, 0.002497, 0.54845, 0.964839]
[0.24578, 0.13368, 0.704378, 0.80709, 0.748982, 0.40143]
[0.389062, 0.327444, 0.27934, 0.140607, 0.299915, 0

In [7]:
from get_init_data2 import get_inputs, get_outputs

X_init2 = get_inputs(f_num)
y_init2 = get_outputs(f_num)
# print(X_init2)
# print(y_init2)

In [9]:
X = np.concatenate((X_init, X_q, X_init2), axis=0)
y = np.concatenate((y_init, y_q, y_init2), axis=0)

print(X.shape)
print("Input : Output")
for i, v in enumerate(X):
    print(str(v) + ":" + str(y[i]))
# print("INPUTS")
# print(X)
# print("OUTPUTS")
# print(y)

(76, 4)
Input : Output
[0.8969810543559719 0.7256279703482243 0.17540430890241154
 0.7016943693717497]:-22.108287785435817
[0.8893563957199797 0.49958785537696504 0.5392688577643431
 0.5087834390633533]:-14.601396631953161
[0.25094624275233424 0.03369313054735357 0.14538002473552147
 0.49493242056497855]:-11.699932463834426
[0.34696206091432413 0.006250400244917853 0.7605636064368956
 0.6130235572343078]:-16.053765110568296
[0.1248711812207095 0.12977019306510673 0.38440048300130336
 0.28707610106061277]:-10.069633426012889
[0.8013027073487429 0.5002310937156627 0.7066445599562093
 0.19510284101191622]:-15.487082537755288
[0.2477082618994807 0.060445427321989764 0.04218634513719777
 0.44132425054403157]:-12.68168497681565
[0.7467022420948025 0.7570915044306501 0.3693530595537917
 0.20656628103766395]:-16.02639976898352
[0.400665027194423 0.07257425107127335 0.8867682538447798
 0.24384228978922717]:-17.049234647535723
[0.6260705961944015 0.5867512592598441 0.4388057821707446
 0.77885769

In [11]:
from ba_optimizer_v1 import BayesianOptimizer

optimizer = BayesianOptimizer(np.array(X, dtype=np.float64), y, bounds=(0, 1))

# Re-optimize with updated data
new_submission = optimizer.optimize_step(
    num_candidates=1000,
    acquisition_func='ucb',
    kappa=2.0
)

print(f"\nNEW RECOMMENDATION: {new_submission['best_point']}")
print(f"UCB Score: {new_submission['best_acquisition']:.4f}")

BAYESIAN OPTIMIZATION STEP
1. Computing function metrics...
   Current best: -4.0255
   Dataset size: 76
   Estimated noise: 0.0000

2. Updating surrogate model...
GP fitted with kernel: 3.16**2 * RBF(length_scale=1.05) + WhiteKernel(noise_level=0.00358)
Log-marginal likelihood: 17.2718

3. Generating 1000 candidate samples...
4. Computing UCB acquisition function...
5. Selecting best points for submission...

RECOMMENDED NEXT POINT:
Location: [0.43313299 0.32163532 0.39399564 0.28949277]
Acquisition: -1.9200
GP Prediction: -2.9078 ± 0.4939

NEW RECOMMENDATION: [0.43313299 0.32163532 0.39399564 0.28949277]
UCB Score: -1.9200


In [13]:

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel

kernel = ConstantKernel(1.0, (1e-3, 1e3)) * RBF(length_scale=0.1)

gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=15)

gpr.fit(X, y)

GaussianProcessRegressor(kernel=1**2 * RBF(length_scale=0.1),
                         n_restarts_optimizer=15)

In [15]:
###### Acquisition Function - Expected Improvement#########
from scipy.stats import norm

x_range = np.linspace(0, 1, 40)

dim = 4
X_grid = np.meshgrid(*([x_range] * dim))
X_grid = np.stack(X_grid, axis=-1).reshape(-1, dim)


def compute_expected_improvement(x):
    mu, sigma = gpr.predict([x], return_std=True)
    y_max = np.max(y)
    z = (mu - y_max) / sigma
    ei = (mu - y_max) * norm.cdf(z) + sigma * norm.pdf(z)
    return ei

ei_values = [compute_expected_improvement(x) for x in X_grid]

next_idx = np.argmax(ei_values)
EI_NextQuery = X_grid[next_idx]

print("Expected Improvement - Next Query: ", EI_NextQuery)


KeyboardInterrupt: 

In [18]:
from ba_optimizer_v2 import BayesianOptimizer as BayesianOptimizer_v2
from sklearn.gaussian_process.kernels import Matern, ConstantKernel, WhiteKernel

X = np.array(X, dtype=np.float64)

optimizer_v2 = BayesianOptimizer_v2(X, y, bounds=(0, 1))

kernel = ConstantKernel(1.0, (1e-3, 1e3)) * RBF(length_scale=0.1)
new_submission = optimizer_v2.optimize_step(
    num_candidates=10000,
    acquisition_func='ucb',
    kappa=1.0, kernel=kernel
)

print(f"\n optimizer_v2 NEW RECOMMENDATION: {new_submission['best_point']}")
print(f"optimizer_v2 UCB Score: {new_submission['best_acquisition']:.4f}")

BAYESIAN OPTIMIZATION STEP
1. Computing function metrics...
   Current best: -4.0255
   Dataset size: 76
   Estimated noise: 0.0000

2. Updating surrogate model...
GP fitted with kernel: 1.81**2 * RBF(length_scale=0.535)
Log-marginal likelihood: 17.4557

3. Generating 10000 candidate samples...
4. Computing UCB acquisition function...
5. Selecting best points for submission...

RECOMMENDED NEXT POINT:
Location: [0.41326355 0.32572635 0.43885842 0.39134569]
Acquisition: -1.0119
GP Prediction: -1.3069 ± 0.2950

 optimizer_v2 NEW RECOMMENDATION: [0.41326355 0.32572635 0.43885842 0.39134569]
optimizer_v2 UCB Score: -1.0119


# Calcualting after 27 May outputs

In [21]:
input_may_27 = np.array([0.435897, 0.333333, 0.410256, 0.410256])
output_may_27 = np.float64(0.2278162892112161)

X = np.append(X, np.array([input_may_27], dtype=np.float64), axis=0)
y = np.append(y, output_may_27)

y_max = np.max(y)
print("y_max is ", y_max, "at: ", X[np.where(y == y_max)][0])

y_max is  0.2278162892112161 at:  [0.435897 0.333333 0.410256 0.410256]


In [23]:
optimizer_v2.add_observation(input_may_27, output_may_27)

optimizer_v2 = BayesianOptimizer_v2(X, y, bounds=(0, 1))
optimizer_v2.optimize_step(
    num_candidates=10000,
    acquisition_func='ucb',
    kernel=kernel)

Added observation: [0.435897 0.333333 0.410256 0.410256] -> 0.2278
BAYESIAN OPTIMIZATION STEP
1. Computing function metrics...
   Current best: 0.2278
   Dataset size: 77
   Estimated noise: 0.0000

2. Updating surrogate model...
GP fitted with kernel: 1.52**2 * RBF(length_scale=0.483)
Log-marginal likelihood: 14.0165

3. Generating 10000 candidate samples...
4. Computing UCB acquisition function...
5. Selecting best points for submission...

RECOMMENDED NEXT POINT:
Location: [0.52554925 0.3115845  0.38624136 0.4692938 ]
Acquisition: 0.5338
GP Prediction: -0.2131 ± 0.3734


{'best_point': array([0.52554925, 0.3115845 , 0.38624136, 0.4692938 ]),
 'best_acquisition': 0.5337532553808609,
 'top_k_points': array([[0.52554925, 0.3115845 , 0.38624136, 0.4692938 ],
        [0.53948549, 0.30998467, 0.38651044, 0.46488977],
        [0.47300228, 0.21148818, 0.37677318, 0.4575316 ],
        [0.43703414, 0.32932012, 0.33245164, 0.41249717],
        [0.41326355, 0.32572635, 0.43885842, 0.39134569]]),
 'top_k_acquisitions': array([0.53375326, 0.46923665, 0.40300491, 0.36597393, 0.22613684]),
 'gp_predictions': (array([-0.2131096 , -0.33129372, -0.46904923, -0.03028436,  0.0279529 ]),
  array([0.37343143, 0.40026518, 0.43602707, 0.19812914, 0.09909197])),
 'current_best_observed': 0.2278162892112161,
 'current_best_point': array([0.435897, 0.333333, 0.410256, 0.410256])}

In [25]:
optimizer_v2.optimize_step(
    num_candidates=10000,
    acquisition_func='ei',
    kernel=kernel)

BAYESIAN OPTIMIZATION STEP
1. Computing function metrics...
   Current best: 0.2278
   Dataset size: 77
   Estimated noise: 0.0000

2. Updating surrogate model...
GP fitted with kernel: 1.52**2 * RBF(length_scale=0.483)
Log-marginal likelihood: 14.0165

3. Generating 10000 candidate samples...
4. Computing EI acquisition function...
5. Selecting best points for submission...

RECOMMENDED NEXT POINT:
Location: [0.52554925 0.3115845  0.38624136 0.4692938 ]
Acquisition: 0.0218
GP Prediction: -0.2131 ± 0.3734


{'best_point': array([0.52554925, 0.3115845 , 0.38624136, 0.4692938 ]),
 'best_acquisition': 0.021791531069079238,
 'top_k_points': array([[0.52554925, 0.3115845 , 0.38624136, 0.4692938 ],
        [0.53948549, 0.30998467, 0.38651044, 0.46488977],
        [0.47300228, 0.21148818, 0.37677318, 0.4575316 ],
        [0.43703414, 0.32932012, 0.33245164, 0.41249717],
        [0.50395994, 0.22032685, 0.42717926, 0.46789086]]),
 'top_k_acquisitions': array([0.02179153, 0.01477913, 0.01017683, 0.00896897, 0.00270153]),
 'gp_predictions': (array([-0.2131096 , -0.33129372, -0.46904923, -0.03028436, -0.77129603]),
  array([0.37343143, 0.40026518, 0.43602707, 0.19812914, 0.46685707])),
 'current_best_observed': 0.2278162892112161,
 'current_best_point': array([0.435897, 0.333333, 0.410256, 0.410256])}

# Calcualting after 2 Jun outputs

In [28]:


input_jun_2 = np.array([0.525549, 0.311584, 0.386241, 0.469293])
output_jun_2 = np.float64(-3.0124550087689417)

X = np.append(X, np.array([input_jun_2], dtype=np.float64), axis=0)
y = np.append(y, output_jun_2)

y_max = np.max(y)
print("Max y is ", y_max, "at: ", X[np.where(y == y_max)][0])

Max y is  0.2278162892112161 at:  [0.435897 0.333333 0.410256 0.410256]


In [32]:
optimizer_v2.add_observation(input_jun_2, output_jun_2)

print(f"\n\n optimizer_v2 acquisition_func='ei'")

optimizer_v2.optimize_step(
    num_candidates=10000,
    acquisition_func='ei',
    kernel=kernel)

Added observation: [0.525549 0.311584 0.386241 0.469293] -> -3.0125


 optimizer_v2 acquisition_func='ei'
BAYESIAN OPTIMIZATION STEP
1. Computing function metrics...
   Current best: 0.2278
   Dataset size: 78
   Estimated noise: 0.0000

2. Updating surrogate model...
GP fitted with kernel: 1.17**2 * RBF(length_scale=0.393)
Log-marginal likelihood: 2.4137

3. Generating 10000 candidate samples...
4. Computing EI acquisition function...
5. Selecting best points for submission...

RECOMMENDED NEXT POINT:
Location: [0.41022827 0.42741948 0.40689702 0.35446519]
Acquisition: 0.6167
GP Prediction: 0.8396 ± 0.3399


{'best_point': array([0.41022827, 0.42741948, 0.40689702, 0.35446519]),
 'best_acquisition': 0.616677982510658,
 'top_k_points': array([[0.41022827, 0.42741948, 0.40689702, 0.35446519],
        [0.34497917, 0.44282994, 0.4091304 , 0.41151386],
        [0.43549633, 0.36654722, 0.46831886, 0.32753394],
        [0.41326355, 0.32572635, 0.43885842, 0.39134569],
        [0.44090739, 0.34635915, 0.45306973, 0.38598458]]),
 'top_k_acquisitions': array([0.61667798, 0.61056375, 0.38461806, 0.36846963, 0.24521021]),
 'gp_predictions': (array([0.83964197, 0.80094291, 0.60444693, 0.59626666, 0.46887984]),
  array([0.33990298, 0.52868979, 0.25584168, 0.11423914, 0.15621844])),
 'current_best_observed': 0.2278162892112161,
 'current_best_point': array([0.435897, 0.333333, 0.410256, 0.410256])}